In [19]:
import os
from groq import Groq
from dotenv import load_dotenv
from typing import List, Optional
from prompts import base, content, cot
from pydantic import BaseModel, Field, RootModel

load_dotenv()

True

In [20]:
class Event(BaseModel):
    event_id: str
    title: str
    start_time: str
    end_time: Optional[str] = None
    duration: Optional[int] = None
    host: List[str]
    location: str
    categories: List[str]
    description: Optional[str] = None


class DateEvents(BaseModel):
    date: str
    events: List[Event]


class EventItinerary(BaseModel):
    RootModel: List[DateEvents]


In [21]:
client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)

In [22]:
resp = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": base,
        },
        {
            "role": "user",
            "content": content,
        },
    ],
    response_format={"type": "json_object"},
    temperature=0,
    max_tokens=8000,
)

In [23]:
print(resp)

ChatCompletion(id='chatcmpl-b5baa453-fc58-4904-86f2-34a7080f1786', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n   "E001":{\n      "event_id":"E001",\n      "title":"March Past",\n      "start_time":"5:00 PM",\n      "end_time":"5:30 PM",\n      "duration":30,\n      "host":null,\n      "location":"Football Ground",\n      "categories":null,\n      "description":null\n   },\n   "E002":{\n      "event_id":"E002",\n      "title":"100 m Heats Boys and Girls",\n      "start_time":"5:30 PM",\n      "end_time":"6:00 PM",\n      "duration":30,\n      "host":null,\n      "location":"Football Ground",\n      "categories":null,\n      "description":null\n   },\n   "E003":{\n      "event_id":"E003",\n      "title":"Chess Boys and Girls",\n      "start_time":"5:30 PM",\n      "end_time":"6:30 PM",\n      "duration":60,\n      "host":null,\n      "location":"Amphitheatre",\n      "categories":null,\n      "description":null\n   },\n   "E004

In [24]:
print(resp.choices[0].message.content)

{
   "E001":{
      "event_id":"E001",
      "title":"March Past",
      "start_time":"5:00 PM",
      "end_time":"5:30 PM",
      "duration":30,
      "host":null,
      "location":"Football Ground",
      "categories":null,
      "description":null
   },
   "E002":{
      "event_id":"E002",
      "title":"100 m Heats Boys and Girls",
      "start_time":"5:30 PM",
      "end_time":"6:00 PM",
      "duration":30,
      "host":null,
      "location":"Football Ground",
      "categories":null,
      "description":null
   },
   "E003":{
      "event_id":"E003",
      "title":"Chess Boys and Girls",
      "start_time":"5:30 PM",
      "end_time":"6:30 PM",
      "duration":60,
      "host":null,
      "location":"Amphitheatre",
      "categories":null,
      "description":null
   },
   "E004":{
      "event_id":"E004",
      "title":"Football (Match -1)",
      "start_time":"6:15 PM",
      "end_time":"7:00 PM",
      "duration":45,
      "host":null,
      "location":"Football Ground",
  

In [26]:
json_content = resp.choices[0].message.content

In [32]:
import json

data = json.loads(json_content)

# Check if the data is valid by using the pydantic model
event_itinerary = EventItinerary.model_validate(data)


ValidationError: 1 validation error for EventItinerary
RootModel
  Field required [type=missing, input_value={'E001': {'event_id': 'E0...e, 'description': None}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing